# Auto-generated from `tools/py_to_ipynb.py`

Generated on 2025-11-09T22:01:34.

This notebook was created programmatically to mirror the original Python script.


In [ ]:
from pathlib import Path
import sys
project_root = str(Path.cwd().parent.resolve())
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [ ]:
import json
import re
from datetime import datetime
from pathlib import Path
from typing import Iterable, List


PROJECT_ROOT = Path("/home/tzhang174/EVData_XGame").resolve()
OUTPUT_DIR = PROJECT_ROOT / "ipynb"

# Directories to exclude anywhere in the relative path
EXCLUDED_DIR_NAMES = {
    "ipynb",
    "__pycache__",
    ".git",
    "venv",
    ".venv",
    "env",
    "build",
    "dist",
}


def list_python_files(root: Path) -> List[Path]:
    python_files: List[Path] = []
    for path in root.rglob("*.py"):
        # Skip hidden files/dirs and excluded dirs
        rel_parts = path.relative_to(root).parts
        if any(
            part in EXCLUDED_DIR_NAMES or part.startswith(".")
            for part in rel_parts[:-1]
        ):
            continue
        python_files.append(path)
    return python_files


def rewrite_paths_to_mw_project(code: str) -> str:
    """
    Rewrite string literals (including f-strings) that end with .csv/.json/.png
    to /home/mw/project/<basename>.<ext>, preserving any f-string placeholders
    in the basename portion.
    """
    # Match optional string prefix (e.g., f, r, fr, b), single or double quote,
    # capture the path part (no quotes), ensure it ends with .csv/.json/.png,
    # and close with the same quote. Case-insensitive on the extension.
    pattern = re.compile(
        r"""(?P<prefix>(?:[rRuUbBfF]+)?)
            (?P<q>['"])
            (?P<path>[^'"]*?)
            (?P<ext>\.(?: Union[csv, json]|png))
            (?P=q)""",
        re.VERBOSE | re.IGNORECASE,
    )

    def _replacement(m: re.Match) -> str:
        prefix = m.group("prefix") or ""
        q = m.group("q")
        path_part = m.group("path") or ""
        ext = m.group("ext")
        # Keep only the last segment (basename), preserving any f-string placeholders
        # Split on both / and \ to cover different styles
        segments = re.split(r"[\\/]", path_part)
        basename = segments[-1] if segments else path_part
        new_inner = f"/home/mw/project/{basename}{ext}"
        return f"{prefix}{q}{new_inner}{q}"

    return pattern.sub(_replacement, code)


def build_notebook_cells(source_py_path: Path, transformed_source: str) -> List[dict]:
    rel = source_py_path.relative_to(PROJECT_ROOT)
    header_md = (
        f"# Auto-generated from `{rel.as_posix()}`\n\n"
        f"Generated on {datetime.now().isoformat(timespec='seconds')}.\n\n"
        "This notebook was created programmatically to mirror the original Python script.\n"
    )
    sys_path_code = (
        "from pathlib import Path\n"
        "import sys\n"
        "project_root = str(Path.cwd().parent.resolve())\n"
        "if project_root not in sys.path:\n"
        "    sys.path.insert(0, project_root)\n"
    )
    return [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": header_md,
        },
        {
            "cell_type": "code",
            "metadata": {},
            "execution_count": None,
            "outputs": [],
            "source": sys_path_code,
        },
        {
            "cell_type": "code",
            "metadata": {},
            "execution_count": None,
            "outputs": [],
            "source": transformed_source,
        },
    ]


def make_notebook(source_py_path: Path, transformed_source: str) -> dict:
    return {
        "cells": build_notebook_cells(source_py_path, transformed_source),
        "metadata": {},
        "nbformat": 4,
        "nbformat_minor": 5,
    }


def convert_one(py_path: Path, out_dir: Path) -> Path:
    text = py_path.read_text(encoding="utf-8")
    transformed = rewrite_paths_to_mw_project(text)
    nb = make_notebook(py_path, transformed)
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{py_path.stem}.ipynb"
    out_path.write_text(json.dumps(nb, ensure_ascii=False, indent=1), encoding="utf-8")
    return out_path


def main() -> None:
    py_files = list_python_files(PROJECT_ROOT)
    print(f"Discovered {len(py_files)} Python files under {PROJECT_ROOT}")
    generated: List[Path] = []
    for p in py_files:
        out_path = convert_one(p, OUTPUT_DIR)
        generated.append(out_path)
        print(f"Generated: {out_path}")
    print(f"Done. Wrote {len(generated)} notebooks to {OUTPUT_DIR}")


if __name__ == "__main__":
    main()


